# Лабораторная работа 3. Модель однофазной одноканальной замкнутой системы обслуживания
## Задание 1

In [1]:
variant <- 5
set.seed(variant)

k <- sample(c(4:9), 1)

pp1 <- runif(4)
pp2 <- runif(3)
pp3 <- runif(2)

p1 <- pp1 / sum(pp1)
p2 <- c(c(0), pp2 / sum(pp2))
p3 <- c(c(0, 0), pp3 / sum(pp3))
p4 <- c(0, 0, 0, 1)

P <- data.frame()

P <- rbind(P, p1, p2, p3, p4)

rownames(P) <- c("p1", "p2", "p3", "p4")
colnames(P) <- c("", "", "", "")

View(P)

print(paste("k =", as.character(k)))

,,,,
,<dbl>,<dbl>,<dbl>,<dbl>
p1,0.3441331,0.4604769,0.1428322,0.05255779
p2,0.0000000,0.3441697,0.2591911,0.39663917
p3,0.0000000,0.0000000,0.8964781,0.10352190
p4,0.0000000,0.0000000,0.0000000,1.00000000


[1] "k = 5"


### Численно
Симулируем проход по матрице переходных вероятностей для трех сценариев:
1. k - 2 осмотров,
2. k - 1 осмотров,
3. k осмотров.

Выполнив эту операцию N раз для каждого сценария, получим три вектора,
содержащие N состояний. Это состояния, в которых оставалась модель после
симуляции:
$$
  Scenario_i = \{State_1 ... State_N\}, i \in \{1, 2, 3\}, \\
  State \in \{S1, S2, S3, S4\}
$$

In [2]:
# Получаем значение, соответствующее отрезку.
# v - случайная величина. Если нужно сгенерировать распределение по `p`, нужно
# подать runif(1)
# X - величины, соответствующие отрезкам.
# За отрезки отвечает параметр p, хранящий последовательность длин.
get_value_in_range <- function(v, X, p) {
    border <- 0
    for (i in seq_along(p)) {
        border <- border + p[i]
        if (v < border) {
            return(X[i])
        }
    }
}

In [3]:
make_step <- function(current_state) {
    possible_states <- P[current_state, ]

    return(get_value_in_range(runif(1), 1:4, possible_states))
}

In [4]:
walk <- function(starting_state, times) {
    if (times < 0) {
        return(starting_state)
    }

    next_state <- make_step(starting_state)

    return(walk(next_state, times - 1))
}

In [5]:
N <- 10000
test_walk <- function(first_state, k) {
    results <- c()

    for (i in 1:N) {
        results <- append(results, walk(first_state, k))
    }

    return(results)
}

In [6]:
first_state <- 1 # Выбрали произвольное.

In [7]:
Scenario1 <- test_walk(first_state, k - 2)

In [8]:
Scenario2 <- test_walk(first_state, k - 1)

In [9]:
Scenario3 <- test_walk(first_state, k)

В полученных сценариях посчитаем вероятности получения каждого состояния.

In [10]:
get_probability <- function(States, state) {
    number_of_states <- length(States[States == state])
    number_of_all_states <- length(States)

    return(number_of_states / number_of_all_states)
}

In [11]:
get_probabilities_to_stay <- function(States) {
    return(
        unlist(
            lapply(seq_along(P), function(state) get_probability(States, state))
        )
    )
}

In [12]:
Scenario1Propabilities <- get_probabilities_to_stay(Scenario1)

In [13]:
Scenario2Propabilities <- get_probabilities_to_stay(Scenario2)

In [14]:
Scenario3Propabilities <- get_probabilities_to_stay(Scenario3)

In [15]:
results <- data.frame(
    Scenario1Propabilities,
    Scenario2Propabilities,
    Scenario3Propabilities
)
colnames(results) <- c("k - 2", "k - 1", "k")
rownames(results) <- rownames(P)

results

,k - 2,k - 1,k
,<dbl>,<dbl>,<dbl>
p1,0.0146,0.0050,0.0021
p2,0.0791,0.0345,0.0159
p3,0.3725,0.3600,0.3284
p4,0.5338,0.6005,0.6536


### Теоретически

In [16]:
initial_state_probabilities <- c(1, 0, 0, 0)
transition_matrix <- data.matrix(P)
transition_matrix

,,,,
p1,0.3441331,0.4604769,0.1428322,0.05255779
p2,0.0000000,0.3441697,0.2591911,0.39663917
p3,0.0000000,0.0000000,0.8964781,0.10352190
p4,0.0000000,0.0000000,0.0000000,1.00000000


In [17]:
if (!require("matrixcalc")) {
    install.packages("matrixcalc")
}

initial_state_probabilities %*% matrix.power(transition_matrix, k)

Loading required package: matrixcalc



,,,
0.004826501,0.03229804,0.3579729,0.6049026


Как видно, теоретически вычисленная матрица с некоторой точностью совпадает
со значениями, полученными теоретически. При увеличении количества
экспериментов $N$ точность только увеличивается.

# Задание 2

In [ ]:
variant <- 5
set.seed(variant)
k <- sample(c(10:25), 1)
t1 <- sample(c(14:20), 1)
t2 <- sample(c(2:5), 1)
View(data.frame(k, t1, t2))

In [ ]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

set.seed(42)

env <- simmer("SuperDuperSim")
env

In [ ]:
programmers <- trajectory("programmers' path") %>%
    ## add an intake activity
    seize("server", 1) %>%
    timeout(function() rnorm(1, 5)) %>%
    release("server", 1)

In [ ]:
env %>%
    add_resource("server", 1) %>%
    add_generator("programmers", programmers, function() {
        return(10)
    })
# , function() rnorm(1, 0.1, 2))

In [ ]:
env %>%
    run(50) %>%
    now()
env %>% peek(3)

In [ ]:
env %>% get_n_generated("programmers")

In [ ]:
env %>% get_queue_count("server")

In [ ]:
env %>% get_mon_resources()